In [2]:
import logging
import apiclient
import ssl
import time

logger = logging.getLogger(__name__)

In [5]:
class GDriveError(Exception):
    pass
    

In [6]:
class GDriveNetworkError(RuntimeError):
    pass

In [7]:
def retryer(max_retries=10, timeout=2):
    logger.debug(f'max_retries: {max_retries}, timeout: {timeout}')
    def decorator(func):
        @wraps(func)
        def retry(*args, **kwargs):
            network_exceptions = (
                apiclient.errors.HttpError,
                ssl.SSLEOFError,
            )
            for i in range(max_retries):
                logger.debug(f'attempt: {i}')
                try:
                    result = func(*args, **kwargs)
                except network_exceptions:
                    time.sleep(timeout)
                    continue
                else:
                    return result
            else:
                raise GDriveNetworkError('error making connection')
        return retry
    return decorator



In [25]:
class GoogleDrive():
    pass
    def __init__(self, oath_cred=None):
        logger.debug('create GoogleDrive object')
        self.oath_cred = oauth_cred

# Notes
Googles OAuth 2.0 for installed apps:
https://googleapis.github.io/google-api-python-client/docs/oauth-installed.html

Read more about how to distribute a secrets.json file here:
https://stackoverflow.com/questions/62418441/connecting-a-desktop-app-to-google-photos-without-disclosing-the-apps-secret-ke

In [24]:

from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))


if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'